## scikit-learn sample_weight compliance report

This notebook runs compliance tests on all scikit-learn estimators. Estimator as inspected to check whether they are expected to have a stochastic fit or not. If the fit is stochastic, a dedicated statistical test is performed, otherwise a deterministic estimator check is run instead.

In [1]:
import sklearn

sklearn.show_versions()


System:
    python: 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
executable: /Users/shrutinath/micromamba/envs/scikit-learn/bin/python
   machine: macOS-14.3-arm64-arm-64bit

Python dependencies:
      sklearn: 1.7.dev0
          pip: 24.0
   setuptools: 75.8.0
        numpy: 2.0.0
        scipy: 1.14.0
       Cython: 3.0.10
       pandas: 2.2.2
   matplotlib: 3.9.0
       joblib: 1.4.2
threadpoolctl: 3.5.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 8
         prefix: libopenblas
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libopenblas.0.dylib
        version: 0.3.27
threading_layer: openmp
   architecture: VORTEX

       user_api: openmp
   internal_api: openmp
    num_threads: 8
         prefix: libomp
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libomp.dylib
        version: None


In [2]:
from inspect import signature
import traceback
import warnings
import pandas as pd
from sklearn.utils import all_estimators
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer
import threadpoolctl

import sys
sys.path.insert(1,'../src/')
from sample_weight_audit import check_weighted_repeated_estimator_fit_equivalence
from sample_weight_audit.sklearn_stochastic_params import STOCHASTIC_FIT_PARAMS

# HistGradientBoostingClassifier trashes the OpenMP thread pool on repeated
# small fits.
threadpoolctl.threadpool_limits(limits=1, user_api="openmp")
warnings.filterwarnings("ignore", category=RuntimeWarning)  # division by zero in AdaBoost
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # liblinear can fail to converge
warnings.filterwarnings("ignore", category=UserWarning)  # KBinsDiscretizer with collapsed bins

In [3]:
from sklearn.linear_model import LogisticRegressionCV

ESTIMATORS_TO_SKIP = [
    LogisticRegressionCV,  # too slow and already somewhat tested by LogisticRegression
]

In [ ]:
from sklearn.utils._testing import  assert_allclose_dense_sparse,set_random_state
from sklearn.base import clone
import numpy as np
from sklearn.utils import shuffle
from sklearn.utils.estimator_checks import (
    _enforce_estimator_tags_y,
)

def check_sample_weight_scaling(estimator_orig):
    # check that setting sample_weight to zero / integer is equivalent
    # to removing / repeating corresponding samples.
    estimator_1= clone(estimator_orig())
    estimator_2 = clone(estimator_orig())
    set_random_state(estimator_1, random_state=0)
    set_random_state(estimator_2, random_state=0)

    rng = np.random.RandomState(42)
    n_samples = 15
    X = rng.rand(n_samples, n_samples * 2)
    y = rng.randint(0, 3, size=n_samples)

    X, y = shuffle(X, y, random_state=0)
    y = _enforce_estimator_tags_y(estimator_1, y)

    estimator_1.fit(X, y=y, sample_weight=1)
    estimator_2.fit(X, y=y, sample_weight=2)

    for method in ["predict_proba", "decision_function", "predict", "transform"]:
        if hasattr(estimator_orig, method):
            X_pred1 = getattr(estimator_1, method)(X)
            X_pred2 = getattr(estimator_2, method)(X)
            err_msg = (
                f"Comparing the output of {estimator_orig.__name__}.{method} revealed that fitting "
                "with `sample_weight` is not equivalent to fitting with"
                "`sample_weight*scaling_factor`."
            )
            assert_allclose_dense_sparse(X_pred1, X_pred2, err_msg=err_msg)


In [4]:
N_STOCHASTIC_FITS = 100
N_STATISTICAL_TESTS = 58  # measured by running the script
BONFERRONI_CORRECTION = 1 / N_STATISTICAL_TESTS
TEST_THRESHOLD = 0.05 * BONFERRONI_CORRECTION


statistical_test_results = []
missing_sample_weight_support = []
errors = []


for est_name, est_class in all_estimators(
    type_filter=["classifier", "regressor", "cluster", "transformer"]
):
    if est_class in ESTIMATORS_TO_SKIP:
        print(f"Skipping {est_name}")
        continue

    if "sample_weight" not in signature(est_class.fit).parameters:
        print(f"⚠ {est_name} does not support sample_weight")
        missing_sample_weight_support.append(est_name)
        continue

    elif "random_state" not in signature(est_class().__init__).parameters:
        est = est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))

    try:
        if est_name == "CategoricalNB":
            # This estimator expects ordinal inputs so we need to discretize the input
            # features. This is not really valid but it's the best we can do while
            # keeping the dataset generation process common to all estimators of a
            # given type.
            est = Pipeline(
                steps=[
                    (
                        "kbinsdiscretizer",
                        KBinsDiscretizer(
                            encode="ordinal", quantile_method="averaged_inverted_cdf"
                        ),
                    ),
                    ("est", est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))),
                ]
            )
        else:
            est = est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))
    except TypeError as e:
        print(f"⚠ {est_name} failed to instantiate: {e}")
        continue

    print(f"Evaluating {est}")
    try:
        result = check_weighted_repeated_estimator_fit_equivalence(
            est,
            est_name,
            test_name="kstest",
            n_stochastic_fits=N_STOCHASTIC_FITS,
            random_state=0,
        )
        pass_or_fail = "✅" if result.pvalue > TEST_THRESHOLD else "❌"
        print(f"{pass_or_fail} {est_name}: (pvalue: {result.pvalue:.3f})")
        statistical_test_results.append(result)
    except Exception as e:
        print(f"❌ {est} error with: {e}")
        errors.append((est, e))

results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

⚠ ARDRegression does not support sample_weight
Evaluating AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                    min_weight_fraction_leaf=0.1))


100%|██████████| 100/100 [00:06<00:00, 16.63it/s]


✅ AdaBoostClassifier: (pvalue: 0.702)
Evaluating AdaBoostRegressor(estimator=DecisionTreeRegressor(max_features=0.5,
                                                  min_weight_fraction_leaf=0.1))


100%|██████████| 100/100 [00:04<00:00, 20.93it/s]


✅ AdaBoostRegressor: (pvalue: 0.036)
⚠ AdditiveChi2Sampler does not support sample_weight
⚠ AffinityPropagation does not support sample_weight
⚠ AgglomerativeClustering does not support sample_weight
Evaluating BaggingClassifier(estimator=LogisticRegression())


...

✅ SplineTransformer: (pvalue: 1.000)
⚠ StackingClassifier failed to instantiate: StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
⚠ StackingRegressor failed to instantiate: StackingRegressor.__init__() missing 1 required positional argument: 'estimators'
Evaluating StandardScaler()


100%|██████████| 1/1 [00:00<00:00, 376.64it/s]


✅ StandardScaler: (pvalue: 1.000)
⚠ TSNE does not support sample_weight
⚠ TargetEncoder does not support sample_weight
⚠ TfidfTransformer does not support sample_weight
⚠ TheilSenRegressor does not support sample_weight
⚠ TransformedTargetRegressor does not support sample_weight
⚠ TruncatedSVD does not support sample_weight
⚠ TunedThresholdClassifierCV does not support sample_weight
Evaluating TweedieRegressor()


100%|██████████| 1/1 [00:00<00:00, 584.82it/s]

✅ TweedieRegressor: (pvalue: 1.000)
⚠ VarianceThreshold does not support sample_weight
⚠ VotingClassifier failed to instantiate: VotingClassifier.__init__() missing 1 required positional argument: 'estimators'
⚠ VotingRegressor failed to instantiate: VotingRegressor.__init__() missing 1 required positional argument: 'estimators'


In [8]:
print(
    f"✅ {len([r for r in statistical_test_results if r.pvalue > TEST_THRESHOLD])} "
    "passed the statistical test"
)
print(
    f"❌ {len([r for r in statistical_test_results if r.pvalue <= TEST_THRESHOLD])} "
    "failed the statistical test"
)
print(f"❌ {len(errors)} other errors")
print(
    f"⚠ {len(missing_sample_weight_support)} estimators lack sample_weight "
    "support"
)
results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

✅ 44 passed the statistical test
❌ 15 failed the statistical test
❌ 3 other errors
⚠ 112 estimators lack sample_weight support


## Details on the statistical test results

In [7]:
results_df.sort_values("pvalue")[["estimator_name", "pvalue", "deterministic_predictions"]]

,estimator_name,pvalue,deterministic_predictions
41,NuSVC,2.208761e-59,False
47,RandomForestRegressor,2.208761e-59,False
49,Ridge,2.208761e-59,False
54,SVC,2.208761e-59,False
46,RandomForestClassifier,4.417521e-57,False
2,BaggingClassifier,4.395434e-55,False
25,HistGradientBoostingClassifier,2.900986e-53,False
26,HistGradientBoostingRegressor,3.231288e-37,False
53,SGDRegressor,4.111260e-34,False
3,BaggingRegressor,3.357101e-13,False


## Details on errors

In [9]:
import sys

for est, e in errors:
    print(f"❌ {est}: {e}")
    traceback.print_exception(e, file=sys.stdout)
    print()

❌ IsotonicRegression(): too many indices for array: array is 1-dimensional, but 2 were indexed
Traceback (most recent call last):
  File "/var/folders/h4/l_wtcs4s43j2j9454pvp1c580000gn/T/ipykernel_94798/2504969921.py", line 49, in <module>
    result = check_weighted_repeated_estimator_fit_equivalence(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shrutinath/sample_weight_audit_dist/reports/../src/sample_weight_audit/estimator_check.py", line 177, in check_weighted_repeated_estimator_fit_equivalence
    multifit_over_weighted_and_repeated(
  File "/Users/shrutinath/sample_weight_audit_dist/reports/../src/sample_weight_audit/estimator_check.py", line 473, in multifit_over_weighted_and_repeated
    X_train = _enforce_estimator_tags_X(est, X_train)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shrutinath/sklearn-dev/scikit-learn/sklearn/utils/estimator_checks.py", line 3986, in _enforce_estimator_tags_X
    X = X[:, 0]
        ~^^^^^

## List of estimators with missing sample_weight support

In [8]:
for est_name in missing_sample_weight_support:
    print(est_name)

ARDRegression
AdditiveChi2Sampler
AffinityPropagation
AgglomerativeClustering
BernoulliRBM
Binarizer
Birch
CCA
ClassifierChain
ColumnTransformer
DictVectorizer
DictionaryLearning
FactorAnalysis
FastICA
FeatureAgglomeration
FeatureHasher
FeatureUnion
FixedThresholdClassifier
FunctionTransformer
GaussianProcessClassifier
GaussianProcessRegressor
GaussianRandomProjection
GenericUnivariateSelect
HDBSCAN
HashingVectorizer
IncrementalPCA
Isomap
KNNImputer
KNeighborsClassifier
KNeighborsRegressor
KNeighborsTransformer
KernelCenterer
KernelPCA
LabelBinarizer
LabelEncoder
LabelPropagation
LabelSpreading
Lars
LarsCV
LassoLars
LassoLarsCV
LassoLarsIC
LatentDirichletAllocation
LinearDiscriminantAnalysis
LocallyLinearEmbedding
MaxAbsScaler
MeanShift
MinMaxScaler
MiniBatchDictionaryLearning
MiniBatchNMF
MiniBatchSparsePCA
MissingIndicator
MultiLabelBinarizer
MultiTaskElasticNet
MultiTaskElasticNetCV
MultiTaskLasso
MultiTaskLassoCV
NMF
NearestCentroid
NeighborhoodComponentsAnalysis
Normalizer
Nystroe

## Example interactive inspection of the predictions and scores of a given estimator

In [9]:
results = next(r for r in statistical_test_results if r.estimator_name == "GaussianNB")
results.predictions_repeated.shape

(1, 3000)

In [10]:
results.predictions_weighted.shape

(1, 3000)

In [11]:
import numpy as np
np.abs(results.predictions_repeated - results.predictions_weighted).max()

np.float64(3.9999992207384594e-10)

In [12]:
np.abs(results.scores_repeated - results.scores_weighted).max()

np.float64(3.2652280879119644e-10)

In [13]:
from scipy.stats import kstest
kstest(results.scores_repeated, results.scores_weighted)

KstestResult(statistic=np.float64(1.0), pvalue=np.float64(1.0), statistic_location=np.float64(3.262703314947884), statistic_sign=np.int8(1))